# Temperature-Dependent von Bertalanffy Growth Model for Mako Sharks

This notebook is a Jupyter Notebook adaptation of `scripts/temperature_vbg_model.py`.

It implements temperature-dependent growth modeling for shortfin mako sharks using the von Bertalanffy growth (VBG) equation with Q10 temperature scaling.

Two model variants are implemented:
- **Model S**: Single Q10 for both anabolic and catabolic rates
- **Model D**: Separate Q10_A (anabolic) and Q10_B (catabolic)

Authors: Olaf Smits, Konstantinos Pantelakis, Teun van den Berg  Institution: TU Delft


## Imports


In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.optimize import least_squares
from typing import Tuple, Optional, Dict, Any
import warnings


## Constants and Default Parameters


In [ ]:
# Reference temperature for Q10 scaling (°C)
T_REF = 18.0

# Default length-weight relationship parameters for mako sharks
# W = a * L^b (L in cm, W in kg)
LW_A = 4.4e-6
LW_B = 3.14


## Length–Weight Conversion Functions


In [ ]:
def length_to_mass(length_cm: np.ndarray, a: float = LW_A, b: float = LW_B) -> np.ndarray:
    """Convert fork length (cm) to body mass (kg) using allometric relationship."""
    return a * np.asarray(length_cm) ** b


def mass_to_length(mass_kg: np.ndarray, a: float = LW_A, b: float = LW_B) -> np.ndarray:
    """Convert body mass (kg) to fork length (cm) using allometric relationship."""
    return (np.asarray(mass_kg) / a) ** (1.0 / b)


## Temperature Scaling (Q10 and Arrhenius)


In [ ]:
def q10_scale(T: float, T_ref: float, Q10: float) -> float:
    """Calculate Q10 temperature scaling factor."""
    return Q10 ** ((T - T_ref) / 10.0)


def arrhenius_scale(T: float, T_ref: float, E_a: float, k_B: float = 8.617e-5) -> float:
    """Calculate Arrhenius temperature scaling factor (alternative to Q10)."""
    T_K = T + 273.15
    T_ref_K = T_ref + 273.15
    return np.exp(-E_a / k_B * (1.0 / T_K - 1.0 / T_ref_K))


## VBG Differential Equation Functions


In [ ]:
def vbg_ode(t: float, w: np.ndarray, eta: float, kappa: float) -> np.ndarray:
    """Von Bertalanffy growth ODE (temperature-independent).

    dw/dt = eta * w^(2/3) - kappa * w
    """
    w = np.atleast_1d(w)
    w_safe = np.maximum(w, 1e-10)
    return eta * w_safe ** (2.0 / 3.0) - kappa * w_safe


def vbg_ode_temp_single(
    t: float,
    w: np.ndarray,
    eta0: float,
    kappa0: float,
    Q10: float,
    T: float,
    T_ref: float = T_REF,
) -> np.ndarray:
    """Temperature-dependent VBG ODE - Model S (single Q10)."""
    w = np.atleast_1d(w)
    w_safe = np.maximum(w, 1e-10)

    scale = q10_scale(T, T_ref, Q10)
    eta_T = eta0 * scale
    kappa_T = kappa0 * scale

    return eta_T * w_safe ** (2.0 / 3.0) - kappa_T * w_safe


def vbg_ode_temp_dual(
    t: float,
    w: np.ndarray,
    eta0: float,
    kappa0: float,
    Q10_A: float,
    Q10_B: float,
    T: float,
    T_ref: float = T_REF,
) -> np.ndarray:
    """Temperature-dependent VBG ODE - Model D (dual Q10)."""
    w = np.atleast_1d(w)
    w_safe = np.maximum(w, 1e-10)

    eta_T = eta0 * q10_scale(T, T_ref, Q10_A)
    kappa_T = kappa0 * q10_scale(T, T_ref, Q10_B)

    return eta_T * w_safe ** (2.0 / 3.0) - kappa_T * w_safe


## Analytical Solutions


In [ ]:
def vbg_analytical(t: np.ndarray, eta: float, kappa: float, w0: float) -> np.ndarray:
    """Analytical solution of the VBG differential equation."""
    t = np.asarray(t)
    w_inf_cbrt = eta / kappa
    w0_cbrt = w0 ** (1.0 / 3.0)
    return (w_inf_cbrt + (w0_cbrt - w_inf_cbrt) * np.exp(-kappa * t / 3.0)) ** 3


def asymptotic_mass(eta: float, kappa: float) -> float:
    """Calculate asymptotic (equilibrium) mass: w* = (eta/kappa)^3"""
    return (eta / kappa) ** 3


def asymptotic_mass_temp(
    eta0: float,
    kappa0: float,
    T: float,
    Q10_A: float,
    Q10_B: float,
    T_ref: float = T_REF,
) -> float:
    """Calculate temperature-dependent asymptotic mass."""
    eta_T = eta0 * q10_scale(T, T_ref, Q10_A)
    kappa_T = kappa0 * q10_scale(T, T_ref, Q10_B)
    return (eta_T / kappa_T) ** 3


## Numerical Solution Functions


In [ ]:
def simulate_growth_model_S(
    w0: float,
    t_span: Tuple[float, float],
    t_eval: np.ndarray,
    T: float,
    eta0: float,
    kappa0: float,
    Q10: float,
    T_ref: float = T_REF,
) -> np.ndarray:
    """Simulate growth trajectory using Model S (single Q10)."""
    sol = solve_ivp(
        fun=lambda t, w: vbg_ode_temp_single(t, w, eta0, kappa0, Q10, T, T_ref),
        t_span=t_span,
        y0=[w0],
        t_eval=t_eval,
        method="RK45",
        dense_output=False,
    )
    if not sol.success:
        warnings.warn(f"ODE solver failed: {sol.message}")
    return sol.y[0]


def simulate_growth_model_D(
    w0: float,
    t_span: Tuple[float, float],
    t_eval: np.ndarray,
    T: float,
    eta0: float,
    kappa0: float,
    Q10_A: float,
    Q10_B: float,
    T_ref: float = T_REF,
) -> np.ndarray:
    """Simulate growth trajectory using Model D (dual Q10)."""
    sol = solve_ivp(
        fun=lambda t, w: vbg_ode_temp_dual(t, w, eta0, kappa0, Q10_A, Q10_B, T, T_ref),
        t_span=t_span,
        y0=[w0],
        t_eval=t_eval,
        method="RK45",
        dense_output=False,
    )
    if not sol.success:
        warnings.warn(f"ODE solver failed: {sol.message}")
    return sol.y[0]


def simulate_growth_variable_temp(
    w0: float,
    times: np.ndarray,
    temperatures: np.ndarray,
    eta0: float,
    kappa0: float,
    Q10_A: float,
    Q10_B: float,
    T_ref: float = T_REF,
) -> np.ndarray:
    """Simulate growth with time-varying temperature (Euler method)."""
    times = np.asarray(times)
    temperatures = np.asarray(temperatures)

    w = np.zeros(len(times))
    w[0] = w0

    for i in range(1, len(times)):
        dt = times[i] - times[i - 1]
        T = temperatures[i]

        eta_T = eta0 * q10_scale(T, T_ref, Q10_A)
        kappa_T = kappa0 * q10_scale(T, T_ref, Q10_B)

        dwdt = eta_T * w[i - 1] ** (2.0 / 3.0) - kappa_T * w[i - 1]
        w[i] = max(w[i - 1] + dt * dwdt, 0.0)

    return w


## Model Fitting Functions


In [ ]:
def predict_mass_at_age_S(
    params: np.ndarray,
    data: pd.DataFrame,
    w0: float = 2.5,
    T_ref: float = T_REF,
) -> np.ndarray:
    """Predict mass at observed age for all observations using Model S."""
    eta0, kappa0, Q10 = params
    w_pred = np.zeros(len(data))

    for idx, (_, row) in enumerate(data.iterrows()):
        age = row["age_years"]
        T = row["mean_sst_C"]

        eta_T = eta0 * q10_scale(T, T_ref, Q10)
        kappa_T = kappa0 * q10_scale(T, T_ref, Q10)
        w_pred[idx] = vbg_analytical(age, eta_T, kappa_T, w0)

    return w_pred


def predict_mass_at_age_D(
    params: np.ndarray,
    data: pd.DataFrame,
    w0: float = 2.5,
    T_ref: float = T_REF,
) -> np.ndarray:
    """Predict mass at observed age for all observations using Model D."""
    eta0, kappa0, Q10_A, Q10_B = params
    w_pred = np.zeros(len(data))

    for idx, (_, row) in enumerate(data.iterrows()):
        age = row["age_years"]
        T = row["mean_sst_C"]

        eta_T = eta0 * q10_scale(T, T_ref, Q10_A)
        kappa_T = kappa0 * q10_scale(T, T_ref, Q10_B)
        w_pred[idx] = vbg_analytical(age, eta_T, kappa_T, w0)

    return w_pred


def residuals_model_S(params: np.ndarray, data: pd.DataFrame, w0: float = 2.5, T_ref: float = T_REF) -> np.ndarray:
    w_pred = predict_mass_at_age_S(params, data, w0, T_ref)
    w_obs = data["mass_kg"].values
    return w_pred - w_obs


def residuals_model_D(params: np.ndarray, data: pd.DataFrame, w0: float = 2.5, T_ref: float = T_REF) -> np.ndarray:
    w_pred = predict_mass_at_age_D(params, data, w0, T_ref)
    w_obs = data["mass_kg"].values
    return w_pred - w_obs


def fit_model_S(
    data: pd.DataFrame,
    x0: Optional[np.ndarray] = None,
    bounds: Optional[Tuple] = None,
    w0: float = 2.5,
    T_ref: float = T_REF,
) -> Dict[str, Any]:
    """Fit Model S (single Q10) using nonlinear least squares."""
    if x0 is None:
        x0 = np.array([2.5, 0.3, 2.0])
    if bounds is None:
        bounds = ([0.1, 0.01, 1.0], [10.0, 1.0, 5.0])

    result = least_squares(
        fun=residuals_model_S,
        x0=x0,
        bounds=bounds,
        args=(data, w0, T_ref),
        method="trf",
    )

    params = result.x
    residuals = result.fun
    sse = np.sum(residuals**2)

    w_obs = data["mass_kg"].values
    ss_tot = np.sum((w_obs - np.mean(w_obs)) ** 2)
    r_squared = 1.0 - sse / ss_tot

    w_pred = predict_mass_at_age_S(params, data, w0, T_ref)

    n = len(data)
    k = 3
    sigma2 = sse / n
    log_likelihood = -n / 2 * (np.log(2 * np.pi * sigma2) + 1)
    aic = 2 * k - 2 * log_likelihood
    bic = k * np.log(n) - 2 * log_likelihood

    return {
        "params": params,
        "param_names": ["eta0", "kappa0", "Q10"],
        "residuals": residuals,
        "sse": sse,
        "r_squared": r_squared,
        "predictions": w_pred,
        "aic": aic,
        "bic": bic,
        "n_params": k,
        "n_obs": n,
        "success": result.success,
        "message": result.message,
    }


def fit_model_D(
    data: pd.DataFrame,
    x0: Optional[np.ndarray] = None,
    bounds: Optional[Tuple] = None,
    w0: float = 2.5,
    T_ref: float = T_REF,
) -> Dict[str, Any]:
    """Fit Model D (dual Q10) using nonlinear least squares."""
    if x0 is None:
        x0 = np.array([2.5, 0.3, 2.0, 2.5])
    if bounds is None:
        bounds = ([0.1, 0.01, 1.0, 1.0], [10.0, 1.0, 5.0, 5.0])

    result = least_squares(
        fun=residuals_model_D,
        x0=x0,
        bounds=bounds,
        args=(data, w0, T_ref),
        method="trf",
    )

    params = result.x
    residuals = result.fun
    sse = np.sum(residuals**2)

    w_obs = data["mass_kg"].values
    ss_tot = np.sum((w_obs - np.mean(w_obs)) ** 2)
    r_squared = 1.0 - sse / ss_tot

    w_pred = predict_mass_at_age_D(params, data, w0, T_ref)

    n = len(data)
    k = 4
    sigma2 = sse / n
    log_likelihood = -n / 2 * (np.log(2 * np.pi * sigma2) + 1)
    aic = 2 * k - 2 * log_likelihood
    bic = k * np.log(n) - 2 * log_likelihood

    return {
        "params": params,
        "param_names": ["eta0", "kappa0", "Q10_A", "Q10_B"],
        "residuals": residuals,
        "sse": sse,
        "r_squared": r_squared,
        "predictions": w_pred,
        "aic": aic,
        "bic": bic,
        "n_params": k,
        "n_obs": n,
        "success": result.success,
        "message": result.message,
    }


## Model Comparison


In [ ]:
def compare_models(result_S: Dict, result_D: Dict) -> Dict[str, Any]:
    """Compare Model S and Model D using information criteria."""
    delta_aic = result_D["aic"] - result_S["aic"]
    delta_bic = result_D["bic"] - result_S["bic"]

    # Approximate likelihood ratio test statistic
    chi_sq = 2 * (result_S["sse"] - result_D["sse"]) / (result_D["sse"] / result_D["n_obs"])

    if delta_aic < -2:
        preferred_aic = "Model D"
    elif delta_aic > 2:
        preferred_aic = "Model S"
    else:
        preferred_aic = "No clear preference"

    if delta_bic < -2:
        preferred_bic = "Model D"
    elif delta_bic > 2:
        preferred_bic = "Model S"
    else:
        preferred_bic = "No clear preference"

    return {
        "aic_S": result_S["aic"],
        "aic_D": result_D["aic"],
        "bic_S": result_S["bic"],
        "bic_D": result_D["bic"],
        "delta_aic": delta_aic,
        "delta_bic": delta_bic,
        "r_squared_S": result_S["r_squared"],
        "r_squared_D": result_D["r_squared"],
        "sse_S": result_S["sse"],
        "sse_D": result_D["sse"],
        "preferred_by_aic": preferred_aic,
        "preferred_by_bic": preferred_bic,
        "lrt_chi_sq": chi_sq,
    }


## Sensitivity Analysis


In [ ]:
def sensitivity_temperature(
    eta0: float,
    kappa0: float,
    Q10_A: float,
    Q10_B: float,
    T_range: np.ndarray,
    T_ref: float = T_REF,
) -> Dict[str, np.ndarray]:
    """Analyze sensitivity of asymptotic mass to temperature."""
    T_range = np.asarray(T_range)
    w_star = np.array([asymptotic_mass_temp(eta0, kappa0, T, Q10_A, Q10_B, T_ref) for T in T_range])

    w_star_ref = asymptotic_mass_temp(eta0, kappa0, T_ref, Q10_A, Q10_B, T_ref)
    relative_change = (w_star - w_star_ref) / w_star_ref * 100

    return {
        "temperature": T_range,
        "w_star": w_star,
        "w_star_ref": w_star_ref,
        "relative_change_percent": relative_change,
    }


def sensitivity_Q10(
    eta0: float,
    kappa0: float,
    T: float,
    Q10_range: np.ndarray,
    T_ref: float = T_REF,
    vary: str = "both",
) -> Dict[str, np.ndarray]:
    """Analyze sensitivity of asymptotic mass to Q10 values."""
    Q10_range = np.asarray(Q10_range)

    if vary == "A":
        w_star = np.array([asymptotic_mass_temp(eta0, kappa0, T, Q, 2.5, T_ref) for Q in Q10_range])
    elif vary == "B":
        w_star = np.array([asymptotic_mass_temp(eta0, kappa0, T, 2.0, Q, T_ref) for Q in Q10_range])
    else:
        w_star = np.array([asymptotic_mass_temp(eta0, kappa0, T, Q, Q, T_ref) for Q in Q10_range])

    return {
        "Q10": Q10_range,
        "w_star": w_star,
        "vary": vary,
    }


## Data Loading


In [ ]:
def load_growth_data(filepath: str) -> pd.DataFrame:
    """Load mako shark growth data from CSV file.

    Required columns: shark_id, age_years, mass_kg, mean_sst_C
    """
    data = pd.read_csv(filepath, comment="#")

    required_cols = ["shark_id", "age_years", "mass_kg", "mean_sst_C"]
    missing = [col for col in required_cols if col not in data.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")
    return data


## Reporting Helpers


In [ ]:
def print_fit_summary(result: Dict[str, Any], model_name: str = "Model") -> None:
    print(f"\n{'=' * 60}")
    print(f"{model_name} Fitting Summary")
    print("=" * 60)

    print("\nParameter Estimates:")
    for name, value in zip(result["param_names"], result["params"]):
        print(f"  {name:12s} = {value:.6f}")

    print(f"\nModel Fit Statistics:")
    print(f"  SSE          = {result['sse']:.4f}")
    print(f"  R²           = {result['r_squared']:.4f}")
    print(f"  AIC          = {result['aic']:.2f}")
    print(f"  BIC          = {result['bic']:.2f}")
    print(f"  N parameters = {result['n_params']}")
    print(f"  N observations = {result['n_obs']}")

    if not result["success"]:
        print(f"\n  Warning: {result['message']}")


def print_comparison_summary(comparison: Dict[str, Any]) -> None:
    print("\n" + "=" * 60)
    print("Model Comparison Summary")
    print("=" * 60)

    print("\nFit Statistics:")
    print(
        f"  Model S: R² = {comparison['r_squared_S']:.4f}, AIC = {comparison['aic_S']:.2f}, BIC = {comparison['bic_S']:.2f}"
    )
    print(
        f"  Model D: R² = {comparison['r_squared_D']:.4f}, AIC = {comparison['aic_D']:.2f}, BIC = {comparison['bic_D']:.2f}"
    )

    print("\nModel Selection:")
    print(f"  ΔAIC (D - S) = {comparison['delta_aic']:.2f}")
    print(f"  ΔBIC (D - S) = {comparison['delta_bic']:.2f}")
    print(f"  Preferred by AIC: {comparison['preferred_by_aic']}")
    print(f"  Preferred by BIC: {comparison['preferred_by_bic']}")


## Run: Fit models + sensitivity (loads CSV if available; otherwise runs a demo)


In [ ]:
from pathlib import Path

print("Temperature-Dependent Von Bertalanffy Growth Model")
print("Shortfin Mako Shark (Isurus oxyrinchus)")
print("=" * 60)
print("\nData Sources:")
print("  - Rolim et al. (2020): South Atlantic population")
print("  - Natanson et al. (2006): North Atlantic population")
print("  - Ribot-Carballal et al. (2005): Eastern Pacific (Baja California)")
print("  - Cerna & Licandeo (2009): Southeast Pacific (Chile)")

# Data path relative to repository layout
data_path = Path('data') / 'mako_growth_data.csv'

try:
    data = load_growth_data(str(data_path))
    print(f"\nLoaded {len(data)} observations from published literature")

    if 'region' in data.columns:
        print("\nObservations by region:")
        for region in data['region'].unique():
            n = len(data[data['region'] == region])
            temp = data[data['region'] == region]['mean_sst_C'].iloc[0]
            print(f"  {region}: {n} observations (SST = {temp}°C)")

    print("\n" + "-" * 60)
    print("Fitting Model S (single Q10)...")
    result_S = fit_model_S(data)
    print_fit_summary(result_S, "Model S (Single Q10)")

    print("\n" + "-" * 60)
    print("Fitting Model D (dual Q10)...")
    result_D = fit_model_D(data)
    print_fit_summary(result_D, "Model D (Dual Q10)")

    comparison = compare_models(result_S, result_D)
    print_comparison_summary(comparison)

    print("\n" + "=" * 60)
    print("Temperature Sensitivity Analysis")
    print("=" * 60)

    eta0, kappa0 = result_D['params'][0], result_D['params'][1]
    Q10_A, Q10_B = result_D['params'][2], result_D['params'][3]

    T_range = np.array([14, 16, 18, 20, 22, 24])
    sens = sensitivity_temperature(eta0, kappa0, Q10_A, Q10_B, T_range)

    print("\nAsymptotic mass (w*) at different temperatures:")
    for T, w, change in zip(sens['temperature'], sens['w_star'], sens['relative_change_percent']):
        print(f"  T = {T:5.1f}°C: w* = {w:7.1f} kg ({change:+6.1f}%)")

    print("\n" + "=" * 60)
    print("Climate Change Impact Projections")
    print("=" * 60)

    T_current = T_REF
    w_current = asymptotic_mass_temp(eta0, kappa0, T_current, Q10_A, Q10_B, T_REF)

    for delta_T, scenario in [(0, 'Current'), (2, '2050 (+2°C)'), (4, '2100 (+4°C)')]:
        T = T_current + delta_T
        w = asymptotic_mass_temp(eta0, kappa0, T, Q10_A, Q10_B, T_REF)
        change = (w - w_current) / w_current * 100
        print(f"  {scenario:18s}: w* = {w:7.1f} kg ({change:+6.1f}%)")

except FileNotFoundError:
    print(f"\nData file not found: {data_path}")
    print("Please ensure the data file exists.")

    print("\nRunning demo with literature parameters...")
    eta0, kappa0 = 2.24, 0.30
    Q10_A, Q10_B = 2.0, 2.5

    print(f"\nExample: eta0={eta0}, kappa0={kappa0}, Q10_A={Q10_A}, Q10_B={Q10_B}")

    T_range = np.array([14, 16, 18, 20, 22, 24])
    sens = sensitivity_temperature(eta0, kappa0, Q10_A, Q10_B, T_range)

    print("\nAsymptotic mass (w*) at different temperatures:")
    for T, w, change in zip(sens['temperature'], sens['w_star'], sens['relative_change_percent']):
        print(f"  T = {T:5.1f}°C: w* = {w:7.1f} kg ({change:+6.1f}%)")
